# Hugging Face

## What is Hugging Face and how does it differ from other ML libraries?

Hugging Face provides:
- Pre-trained transformer models for NLP/CV/Audio
- Model Hub for sharing and discovering models
- Datasets library for ML datasets
- Tools for training and fine-tuning models
- Inference API and model deployment
- AutoML capabilities with AutoTrain
- Spaces for ML app deployment

Key differences from other frameworks:
- Focus on transformer architectures
- Largest collection of pre-trained models
- Stronger community and sharing features
- Better standardization across models
- Simpler fine-tuning workflows
- Integrated deployment solutions

## Different kinds of models available:

- Multimodal models
    - Audio-Text-to-Text, Image-Text-to-Text, Visual Question Answering, Document Question Answering, Video-Text-to-Text, Any-to-Any
- Computer Vision models
    - Depth Estimation, Image Classification, Object Detection, Image Segmentation, Text-to-Image, Image-to-Text, Image-to-Image, Image-to-Video, Unconditional Image Generation, Video Classification, Text-to-Video, Zero-Shot Image Classification, Mask Generation, Zero-Shot Object Detection, Text-to-3D, Image-to-3D, Image Feature Extraction, Keypoint Detection
- Natural Language Processing models
    - Text Classification, Token Classification, Table Question Answering, Question Answering, Zero-Shot Classification, Translation, Summarization, Feature Extraction, Text Generation, Text2Text Generation, Fill-Mask, Sentence Similarity, Audio, Text-to-Speech, Text-to-Audio, Automatic Speech Recognition, Audio-to-Audio, Audio Classification, Voice Activity Detection
- Tabular models
    - Tabular Classification, Tabular Regression, Time Series Forecasting
- Reinforcement Learning models
    - Reinforcement Learning, Robotics
- Other models
    - Graph Machine Learning

## How do you install this?

`!pip install transformers datasets evaluate accelerate torch sentencepiece sacremoses -U`

- transformers -> library for all kinds of NLP tasks
- datasets -> library for all kinds of datasets
- evaluate -> library for evaluation of models
- accelerate -> library for distributed training
- torch -> PyTorch library
- sentencepiece -> library for tokenization
- sacremoses -> library for tokenization

In [1]:
# !pip install transformers datasets evaluate accelerate torch sentencepiece sacremoses -U

## How can you use pipelines?

Pipelines are a high-level API for using pre-trained models for common NLP tasks. They are easy to use and require minimal code. You can select a task and a model, and then use the pipeline to perform the task.

Some parameters are `model="distilgpt2"`, `max_length=20`, `num_return_sequences=2`, etc.

In [36]:
import contextlib
import os
import warnings
from transformers import pipeline, logging

In [32]:
# Suppress all output and logs
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
        # Disable Hugging Face logging
        logging.set_verbosity_error()
        
        # Disable Python warnings
        warnings.filterwarnings("ignore")
        # Initialize pipelines
        classifier = pipeline("sentiment-analysis")  # distilbert-base-uncased-finetuned-sst-2-english model is used by default
        zero_shot = pipeline("zero-shot-classification")
        generator = pipeline("text-generation", model="distilgpt2")
        unmasker = pipeline("fill-mask", model='distilroberta-base')
        ner = pipeline("ner", grouped_entities=True)  # Named Entity Recognition
        qa = pipeline("question-answering")
        summarizer = pipeline("summarization")
        translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

print('classifier: ', classifier(["I love Transformers!", "I hate bugs."]))
print('zero_shot: ', zero_shot("This is a course about NLP models.", candidate_labels=["education", "technology", "sports"]))
print('generator: ', generator("Transformers are great for", max_length=20, num_return_sequences=2))
print('unmasker: ', unmasker("Hugging Face is a <mask> library.", top_k=2))
print('ner: ', ner("Hugging Face is based in New York City."))
print('qa: ', qa(question="Where is Hugging Face based?", context="Hugging Face is based in New York City."))
print('summarizer: ', summarizer("Hugging Face creates tools for NLP. These tools are widely used in AI and ML applications.", min_length=6, max_length=10))
print('translator: ', translator("Hugging Face est une bibliothèque populaire pour le NLP."))

classifier:  [{'label': 'POSITIVE', 'score': 0.9998069405555725}, {'label': 'NEGATIVE', 'score': 0.9967179894447327}]
zero_shot:  {'sequence': 'This is a course about NLP models.', 'labels': ['technology', 'education', 'sports'], 'scores': [0.9376305937767029, 0.05547419190406799, 0.006895218510180712]}
generator:  [{'generated_text': 'Transformers are great for the work done, and I believe it’s all a testament to'}, {'generated_text': 'Transformers are great for the first time, and are not in the same category as their respective consoles'}]
unmasker:  [{'score': 0.176646888256073, 'token': 481, 'token_str': ' free', 'sequence': 'Hugging Face is a free library.'}, {'score': 0.07091348618268967, 'token': 285, 'token_str': ' public', 'sequence': 'Hugging Face is a public library.'}]
ner:  [{'entity_group': 'ORG', 'score': np.float32(0.8907568), 'word': 'Hugging Face', 'start': 0, 'end': 12}, {'entity_group': 'LOC', 'score': np.float32(0.9991805), 'word': 'New York City', 'start': 25, 'e

## How do you load and use pre-trained models?

In [34]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoModelForQuestionAnswering,
    pipeline
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Basic tokenization and inference
inputs = tokenizer("Hello, world!", return_tensors="pt")
outputs = model(**inputs)

In [17]:
import torch
# Specific task models
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
classifier = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
    num_labels=2
)
inputs = tokenizer("I love this movie", return_tensors="pt")

outputs = classifier(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Display results
labels = ["Negative", "Positive"]
result = {labels[i]: float(predictions[0][i]) for i in range(len(labels))}
print(result)

{'Negative': 0.0001234115188708529, 'Positive': 0.9998766183853149}


In [18]:
ner_model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=9
)
entities = ner("My name is Sarah and I live in London")
display(entities)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'I-PER',
  'score': 0.9982994,
  'index': 4,
  'word': 'Sarah',
  'start': 11,
  'end': 16},
 {'entity': 'I-LOC',
  'score': 0.9983972,
  'index': 9,
  'word': 'London',
  'start': 31,
  'end': 37}]

In [19]:
qa_model = AutoModelForQuestionAnswering.from_pretrained(
    'bert-base-uncased'
)
result = qa(question="Who was Jim Henson?",
           context="Jim Henson was a puppeteer")
display(result)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.8786673545837402, 'start': 15, 'end': 26, 'answer': 'a puppeteer'}

## How do you handle tokenization?

In [21]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [22]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [23]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
display(tokenizer)

# Basic tokenization
tokens = tokenizer.tokenize("Hello, how are you?")
display(tokens)
input_ids = tokenizer.encode("Hello, how are you?")
display(input_ids)

# Batch tokenization
inputs = tokenizer(
    ["Hello, how are you?", "I'm fine, thanks!"],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"  # or 'tf' for TensorFlow
)
display(inputs)

# Access different components
input_ids = inputs['input_ids']
display(input_ids)
attention_mask = inputs['attention_mask']
display(attention_mask)
token_type_ids = inputs['token_type_ids']  # for some models
display(token_type_ids)

# Special tokens
cls_token_id = tokenizer.cls_token_id
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
print(cls_token_id, sep_token_id, pad_token_id)

# Decode tokens back to text
text = tokenizer.decode(input_ids[0])
display(text)

long_text = "This is a very long text that exceeds the model's maximum input length. " * 50

# Handle long sequences
inputs = tokenizer(
    long_text,
    max_length=512,
    truncation=True,
    stride=128,
    return_overflowing_tokens=True
)
display(inputs)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

['hello', ',', 'how', 'are', 'you', '?']

[101, 7592, 1010, 2129, 2024, 2017, 1029, 102]

{'input_ids': tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102,    0],
        [ 101, 1045, 1005, 1049, 2986, 1010, 4283,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}

tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102,    0],
        [ 101, 1045, 1005, 1049, 2986, 1010, 4283,  999,  102]])

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]])

101 102 0


'[CLS] hello, how are you? [SEP] [PAD]'

{'input_ids': [[101, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 2003, 1037, 2200, 2146, 3793, 2008, 23651, 1996, 2944, 1005, 1055, 4555, 7953, 3091, 1012, 2023, 200

## How do you implement fine-tuning?

In [ ]:
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
import datasets

# Load dataset
dataset = datasets.load_dataset('imdb')

# Prepare data
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train model
trainer.train()

# Save model
trainer.save_model("./fine_tuned_model")

## How do you work with the datasets library?

In [ ]:
from datasets import (
    load_dataset,
    Dataset,
    DatasetDict,
    Features,
    Value,
    ClassLabel
)

# Load existing dataset
dataset = load_dataset('imdb')
squad_dataset = load_dataset('squad')

# Create custom dataset
data = {
    'text': ["Hello", "World"],
    'label': [0, 1]
}

features = Features({
    'text': Value('string'),
    'label': ClassLabel(num_classes=2, names=['neg', 'pos'])
})

dataset = Dataset.from_dict(data, features=features)

# Dataset operations
# Filter
filtered = dataset.filter(lambda x: len(x['text']) > 100)

# Map
def uppercase(example):
    return {'text': example['text'].upper()}

dataset = dataset.map(uppercase, batched=True)

# Shuffle and select
shuffled = dataset.shuffle(seed=42)
subset = dataset.select(range(100))

# Split dataset
train_test = dataset.train_test_split(test_size=0.2)

# Save and load
dataset.save_to_disk('path/to/dataset')
loaded = Dataset.load_from_disk('path/to/dataset')

# Stream large datasets
streamed_dataset = load_dataset('large_dataset', streaming=True)
for example in streamed_dataset:
    process_example(example)

## How do you handle model evaluation?

In [ ]:
from transformers import TrainerCallback
from evaluate import load

# Load evaluation metric
metric = load("accuracy")
f1_metric = load("f1")

# Custom evaluation function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

# Add to trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Custom evaluation loop
model.eval()
for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)
        predictions = outputs.logits.argmax(-1)
        metric.add_batch(
            predictions=predictions,
            references=batch["labels"]
        )

final_score = metric.compute()

# Custom callback for logging
class EvaluationCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        print(f"Step {state.global_step}: {metrics}")

trainer.add_callback(EvaluationCallback())

## How do you implement custom training loops?

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import get_scheduler

# Initialize optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        print(f"Loss: {loss.item()}")

## How do you use Accelerate for distributed training?

In [ ]:
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed

# Initialize accelerator
accelerator = Accelerator()
logger = get_logger(__name__)

# Prepare model, dataloaders, optimizer
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        optimizer.zero_grad()

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

# Save model
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
accelerator.save(unwrapped_model.state_dict(), "./model_state.pt")

## How do you deploy models?

In [ ]:
# Local deployment with pipeline
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="fine_tuned_model",
    tokenizer="fine_tuned_model"
)

# Export to ONNX
from transformers.onnx import export
from pathlib import Path

export(
    tokenizer=tokenizer,
    model=model,
    output=Path("model.onnx"),
    opset=12
)

# Optimize with ONNX Runtime
import onnxruntime as ort

session = ort.InferenceSession("model.onnx")
outputs = session.run(None, {"input_ids": input_ids})

# Export to TorchScript
traced_model = torch.jit.trace(
    model,
    (input_ids, attention_mask)
)
torch.jit.save(traced_model, "model.pt")

# Gradio interface
import gradio as gr

def predict(text):
    results = classifier(text)
    return results[0]['label'], results[0]['score']

iface = gr.Interface(
    fn=predict,
    inputs="text",
    outputs=["label", "number"]
)
iface.launch()

## How do you use the Hub?

In [ ]:
from huggingface_hub import (
    HfApi,
    Repository,
    create_repo,
    upload_file
)

# Initialize API
api = HfApi()

# Create repository
create_repo("my-model")

# Clone repository
repo = Repository("path/to/local/repo", "username/my-model")
repo.git_pull()

# Push to hub
api.upload_file(
    path_or_fileobj="model.pt",
    path_in_repo="model.pt",
    repo_id="username/my-model"
)

# Upload model
model.push_to_hub("username/my-model")
tokenizer.push_to_hub("username/my-model")

# Download from hub
api.snapshot_download(
    repo_id="username/my-model",
    revision="main"
)

# Model cards
from huggingface_hub import ModelCard

card_content = """
---
language: en
tags:
- sentiment-analysis
- bert
---
# Model Card for my-model
"""

card = ModelCard(card_content)
card.push_to_hub("username/my-model")

## How do you use AutoTrain?

In [ ]:
from autotrain.cli import AutoTrain

# Initialize AutoTrain project
project = AutoTrain(
    project_name="my_project",
    task="text_classification",
    model_name="bert-base-uncased",
    training_data="path/to/train.csv",
    validation_data="path/to/valid.csv"
)

# Configure training
project.config(
    num_epochs=3,
    learning_rate=2e-5,
    batch_size=16,
    max_seq_length=128
)

# Start training
project.train()

# Get best model
best_model = project.get_best_model()

# Deploy model
project.deploy()

## How do you implement text generation?

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TextGenerationPipeline
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Basic generation
inputs = tokenizer("Once upon a time", return_tensors="pt")
outputs = model.generate(**inputs)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Advanced generation parameters
outputs = model.generate(
    **inputs,
    max_length=100,
    num_beams=5,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    do_sample=True,
    num_return_sequences=3
)

# Using pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
results = generator(
    "Once upon a time",
    max_length=100,
    num_return_sequences=3
)

# Getting Started with Hugging Face Transformers: A Beginner's Guide

## Introduction to Hugging Face 🤗

Hugging Face has become the go-to platform for modern machine learning, particularly for transformer models. Think of it as the "GitHub for AI" - offering tools that simplify every stage of the machine learning lifecycle. Here's what makes it special:

**Key Features:**
- **Model Hub**: 200,000+ pre-trained models (NLP, CV, audio)
- **Datasets Library**: 20,000+ ready-to-use datasets
- **Training Tools**: Simple fine-tuning workflows
- **Deployment Solutions**: One-click model deployment
- **Community**: Collaborative ecosystem for sharing

**Why Choose Hugging Face?**
1. Standardized interfaces across models
2. Cutting-edge transformer implementations
3. Beginner-friendly high-level APIs
4. Seamless transition from prototype to production

## Installation 🛠️

Start with these essential packages:
```bash
pip install transformers datasets evaluate accelerate huggingface_hub
```

## Your First Model: Using Pipelines 🚀

The easiest way to get started is with pipelines:

```python
from transformers import pipeline

# Sentiment analysis
sentiment_analyzer = pipeline("sentiment-analysis")
print(sentiment_analyzer("I love coding with Transformers!"))

# Question answering
qa_pipeline = pipeline("question-answering")
answer = qa_pipeline(
    question="What is Hugging Face?",
    context="Hugging Face is a platform for machine learning models."
)
print(answer)
```

Pipelines handle everything automatically:
- Text preprocessing
- Model inference
- Output post-processing

## Understanding Tokenization 🔠

Tokenization converts raw text to model inputs:

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "Transformers are amazing!"
tokens = tokenizer.tokenize(text)
# ['transformers', 'are', 'amazing', '!']

encoded_input = tokenizer(
    text,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"  # PyTorch tensors
)

print(encoded_input)
# {'input_ids': tensor([...]), 'attention_mask': tensor([...])}
```

**Key Tokenization Features:**
- Automatic padding/truncation
- Multiple framework support (PyTorch, TensorFlow)
- Special tokens (CLS, SEP)
- Batch processing

## Working with Datasets 📊

The datasets library makes data handling efficient:

```python
from datasets import load_dataset

# Load a dataset
dataset = load_dataset("imdb")

# Create a custom dataset
from datasets import Dataset
data = {"text": ["Great movie!", "Terrible plot"], "label": [1, 0]}
dataset = Dataset.from_dict(data)

# Preprocess dataset
def preprocess(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True
    )

tokenized_dataset = dataset.map(preprocess, batched=True)

# Split data
train_test = tokenized_dataset.train_test_split(test_size=0.2)
```

**Dataset Features:**
- Streaming for large datasets
- Built-in versioning
- Memory-efficient operations
- Integration with Pandas

## Fine-Tuning Models 🎯

Adapt pre-trained models to your task:

```python
from transformers import (
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

# Start training!
trainer.train()
```

## Evaluating Model Performance 📈

Track metrics during training:

```python
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    return accuracy_metric.compute(
        predictions=predictions,
        references=labels
    )

# Add to trainer
trainer = Trainer(
    ...,
    compute_metrics=compute_metrics
)
```

Common evaluation metrics:
- Accuracy (classification)
- F1 Score (imbalanced data)
- BLEU (translation)
- ROUGE (summarization)

## Custom Training Loops 🔄

For more control:

```python
import torch
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
```

## Deploying Your Model 🚢

Share your model with the world:

**Option 1: Hugging Face Hub**
```python
# Push to hub
model.push_to_hub("my-awesome-model")
tokenizer.push_to_hub("my-awesome-model")
```

**Option 2: Create an API**
```python
# Using Gradio
import gradio as gr

def predict(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return "Positive" if outputs.logits.argmax() == 1 else "Negative"

gr.Interface(
    fn=predict,
    inputs="textbox",
    outputs="label"
).launch()
```

## Advanced Features 🚀

**Distributed Training with Accelerate**
```python
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, dataloader = accelerator.prepare(
    model, optimizer, dataloader
)

for batch in dataloader:
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)
    optimizer.step()
    optimizer.zero_grad()
```

**Text Generation**
```python
generator = pipeline("text-generation", model="gpt2")
generated = generator(
    "Once upon a time",
    max_length=100,
    temperature=0.7,
    num_return_sequences=3
)
```

## Best Practices 💡

1. Start with pipelines for quick prototypes
2. Always use the matching tokenizer for your model
3. Monitor training with TensorBoard
4. Use model cards for documentation
5. Experiment with different learning rates (2e-5 to 5e-5 is common)

## Next Steps 🌟

1. Explore the Model Hub for task-specific models
2. Try fine-tuning on your own dataset
3. Experiment with different architectures (BERT, GPT, T5)
4. Join the Hugging Face community forum
5. Check out the official documentation for advanced techniques

Remember: The key to mastering Hugging Face is practice. Start small, experiment often, and don't hesitate to use the community resources!